# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,21.86,57,20,2.57,US,1722883005
1,1,termas de rio hondo,-27.4998,-64.8604,22.74,42,8,2.95,AR,1722883006
2,2,fort mcmurray,56.7268,-111.3810,22.73,43,0,3.60,CA,1722882779
3,3,iskateley,67.6803,53.1512,6.48,83,3,5.32,RU,1722883008
4,4,tual,-5.6667,132.7500,25.25,85,97,9.29,ID,1722883009


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_vacay_df = city_data_df[((city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)) & ((city_data_df['Wind Speed'] < 4.5) | (city_data_df['Cloudiness'] == 0))]

# Drop any rows with null values
ideal_vacay_df.dropna()

# Display sample data
ideal_vacay_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,21.86,57,20,2.57,US,1722883005
1,1,termas de rio hondo,-27.4998,-64.8604,22.74,42,8,2.95,AR,1722883006
2,2,fort mcmurray,56.7268,-111.3810,22.73,43,0,3.60,CA,1722882779
11,11,shyryayeve,47.3805,30.1964,24.62,29,100,1.36,UA,1722883017
16,16,badger,64.8000,-147.5333,22.03,58,20,2.57,US,1722883024


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,college,64.8569,-147.8028,21.86,57,20,2.57,US,1722883005,
1,1,termas de rio hondo,-27.4998,-64.8604,22.74,42,8,2.95,AR,1722883006,
2,2,fort mcmurray,56.7268,-111.3810,22.73,43,0,3.60,CA,1722882779,
3,3,iskateley,67.6803,53.1512,6.48,83,3,5.32,RU,1722883008,
4,4,tual,-5.6667,132.7500,25.25,85,97,9.29,ID,1722883009,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Define the categories variable
categories = "accommodation.hotel"
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": categories,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Update the bias parameter with the correct format based on current latitude and longitude
    bias = f"proximity:{lon},{lat}"
    params["bias"] = bias

    name_address = requests.get(base_url, params=params)
    name_address_json = name_address.json()

    try:
        hotel_name = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError) as e:
        print(f"Error finding hotel for {hotel_df.loc[index, 'City']}: {e}")
        hotel_name = "No hotel found"

    hotel_df.loc[index, "Hotel Name"] = hotel_name

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_name}")

# Display the DataFrame with the hotel names only
hotel_df.head()

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Starting hotel search
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
termas de rio hondo - nearest hotel: Hotel Panamericano
fort mcmurray - nearest hotel: Nomad Hotel and Suites
iskateley - nearest hotel: Авантаж
tual - nearest hotel: Suita
Error finding hotel for paragominas: 'name'
paragominas - nearest hotel: No hotel found
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
selfoss - nearest hotel: Miðgarður
broken hill - nearest hotel: Ibis Styles
port-aux-francais - nearest hotel: Keravel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
Error finding hotel for shyryayeve: 'name'
shyryayeve - nearest hotel: No hotel found
sinop - nearest hotel: Yıldız Country Hotel Sinop
hassi messaoud - nearest hotel: AMC
olonkinbyen - nearest hotel: Hótel Norðurljós
albany - nearest hotel: Tru
badger - nearest hotel: Clarion Hotel & Suites
mikun' - nearest hotel: Саквояж
adamstown - nearest hotel: Chez Bianca & Benoit
kavieng - nearest hotel: Kavieng Hotel
fisterr

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)